In [7]:
from tsdb_client import TSDBClient
import pandas as pd

In [2]:
cli = TSDBClient(
    host="128.110.25.99",
    port=5432,
    user="chiubj",
    password="bunnygood",
    db="accountdb"
)

In [10]:
strategy_df = cli.execute_query('''
SELECT * FROM dealer.strategy
''', 
    out_type='df')

In [21]:
cli.execute_query('''
    select 
        t_st.name as 策略, 
        t_pos.code as 代碼,
        t_pos.position as 部位,
        t_pos.avg_price as 成本價,
        t_quote.close as 收盤價,
        (position * (close - avg_price))*1000 as 損益, 
        (close / avg_price - 1) * 100 as "損益(%)",
        t_pos.trade_date

    from (
        select 
            strategy, 
            code,
            trade_date,
            sum((case when action = 'B' then 1 else -1 end) * qty) as position,
            sum(price*qty) / sum(qty) as avg_price
        from dealer.trades
        group by strategy, code, trade_date
    )t_pos
left join public.quote_snapshots t_quote on t_pos.code = t_quote.code
left join dealer.strategy t_st on t_pos.strategy = t_st.id 

order by t_pos.strategy, t_pos.code
''', 
    out_type='df')

,策略,代碼,部位,成本價,收盤價,損益,損益(%),trade_date
0,法說會前主力蠢蠢欲動,1504,13,48.70,44.95,-48750.0,-7.700205,2023-05-09
1,法說會前主力蠢蠢欲動,2412,5,126.50,126.00,-2500.0,-0.395257,2023-05-05
2,法說會前主力蠢蠢欲動,3706,20,28.25,27.85,-8000.0,-1.415929,2023-05-11
3,法說會前主力蠢蠢欲動,6414,2,257.50,255.50,-4000.0,-0.776699,2023-05-05
4,注意股10日多,3479,2,119.00,116.00,-6000.0,-2.521008,2023-05-09


In [4]:
strategy_df

,id,name,add_date,status,leverage_ratio,expected_mdd,expected_daily_return
0,1,法說會前主力蠢蠢欲動,2023-05-05,True,0.77,None,None
1,2,注意股10日多,2023-05-05,True,0.41,None,None
2,5,營收首次創新高,2023-05-05,True,0.20,None,None
3,6,籌碼雷達5日多,2023-05-05,True,0.69,None,None
4,3,處置股10日多,2023-05-05,True,0.48,None,None
5,4,開高動能強飆股_1,2023-05-05,True,0.21,None,None


In [5]:
strategy_name = '法說會前主力蠢蠢欲動'

In [8]:
status_df = pd.read_csv(f'status/strategies/{strategy_name}.txt', sep = ' ', header =None)


EmptyDataError: No columns to parse from file